## Exercise 1: Pima Indians Diabetes
In this exercise we will create a neural network to predict the onset of diabetes mellitus in [Pima Indians](https://en.wikipedia.org/wiki/Pima_people), a high risk population who live in southern Arizona.

This type of supervised learning is called binary classification, because our desired output consists into one of two groups.

There are 7 main steps when creating a deep learning model using Keras:

1. Import dependencies
2. Set seed
3. Import data
4. Create model
5. Compile model 
6. Fit model
7. Evaluate model

### Import dependencies
Start by importing the dependencies we will need for the project

In [1]:
from keras.models import Sequential
from keras.layers import Dense
import numpy as np
from sklearn.model_selection import train_test_split
from uoa_mlaas import use_cpu
use_cpu()

Using TensorFlow backend.


### Set seed
Set a seed value so that when we repeatedly run our code we will get the same result. Using the same seed is important when you want to compare algorithms.

In [2]:
seed = 7
np.random.seed(seed)

### Import data
The Pima Indians Diabetes dataset contains 9 features and 768 samples, including whether the subject developed diabetes mellitus within 5 years of the examination (or if it didn't develop). The features in the dataset are described below.

* P: Number of times pregnant 
* PG: Plasma glucose concentration a 2 hours in an oral glucose tolerance test 
* DIA: Diastolic blood pressure (mm Hg) 
* SF: Triceps skin fold thickness (mm) 
* INS: 2-Hour serum insulin (mu U/ml) 
* BMI: Body mass index (weight in kg/(height in m)^2) 
* DP: Diabetes pedigree function 
* Age: (years) 
* Class: whether the subject developed diabetes more than 5 years from examination (0 - diabetes free or 1 - diabetes)

|P|PG|DIA|SF|INS|BMI|DP|Age|Class|
|--|--|--|--|--|--|--|--|--|
|6|148|72|35|0|33.6|0.627|50|1|
|1|85|66|29|0|26.6|0.351|31|0|
|8|183|64|0|0|23.3|0.672|32|1|
|1|89|66|23|94|28.1|0.167|21|0|
|0|137|40|35|168|43.1|2.288|33|1|
|5|116|74|0|0|25.6|0.201|30|0|

To load this data into memory, use the `np.loadtxt` function.

In [3]:
data = np.loadtxt("data/pima.csv", delimiter=",")
print(data)

[[   6.     148.      72.    ...,    0.627   50.       1.   ]
 [   1.      85.      66.    ...,    0.351   31.       0.   ]
 [   8.     183.      64.    ...,    0.672   32.       1.   ]
 ..., 
 [   5.     121.      72.    ...,    0.245   30.       0.   ]
 [   1.     126.      60.    ...,    0.349   47.       1.   ]
 [   1.      93.      70.    ...,    0.315   23.       0.   ]]


Separate the data into input (X) and output (y) variables.

In [4]:
X = data[:, 0:8]
print(X[0])
y = data[:, 8]
print(y[0])

[   6.     148.      72.      35.       0.      33.6      0.627   50.   ]
1.0


Use the `train_test_split` function from scikit-learn to split the input and target data into training and test datasets. A `test_size` of 0.33 is used and the `random_state` is set to our seed so that we get the same results.

In [5]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=seed)

### Create the model
The code snippet below creates a very basic neural network model, with three layers: an input layer, a hidden layer and an output layer.

We start by instantiating a `Sequential` model, onto which we add a number of layers.

The first layer is a fully connected neural network layer, defined by the `Dense` class. The first `Dense` layer added to a `Sequential` model actually acts as two layers: the input layer and the first hidden layer.

The activation function (`activation` parameter) determines the output of a neuron given certain inputs. We use the [Rectifier activation function](https://en.wikipedia.org/wiki/Rectifier_(neural_networks)) (`relu`) within the network because it has good performance.

The first argument in the `Dense` class is the number of neurons in the layer. In the first layer we set this value to 12, which becomes 12 neurons in the hidden layer (choosing a good value for this variable requires trial and error). Our input data set has 8 features, so we set the `input_dim` variable to 8, which creates 8 input neurons, one for each feature.

The last layer is specified by adding a `Dense` layer with 1 neuron. The number of neurons in the output layer always equals the number of outputs you are predicting. In this case we are estimating one output, whether the subject developed diabetes or not, so we set it to 1. The activation function on the last layer is a [Sigmoid activation function](https://en.wikipedia.org/wiki/Sigmoid_function) (`sigmoid`) because we want to guarantee that the output will be between 0 and 1.

In [6]:
model = Sequential()
model.add(Dense(12, input_dim=8, activation='relu', kernel_initializer='normal'))
model.add(Dense(1, activation='sigmoid', kernel_initializer='normal'))

### Compile the model
The next step is to compile the model, by calling the `compile` function. 

The compilation process uses TensorFlow to create a network that can be trained and used to make predictions. Training involves finding weights that can make accurate predictions based on the input data. 

During each iteration of training, the network produces a result, and to check the error, a `loss function` is used to evaluate the effectivness of the neuron weights. This is specified through the `loss` parameter. In this example we use the `binary_crossentropy` loss function, because we are using regression.

How loss functions work is outside the scope of this tutorial, however, you can view the available loss functions in the Keras documentation [here](https://keras.io/losses/).

The last parameter is the `optimizer`, which is the algorithm that actually tries to minimise the error by adjusting the weights in the neural network. The `adam` optimiser uses the [gradient descent algorithm](https://en.wikipedia.org/wiki/Gradient_descent) and is often an efficient choice.

How these algorithms work is also out of scope of the tutorial, however, you can view the available optimizers in the Keras documentation [here](https://keras.io/optimizers/).

In [7]:
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

### Fit the model
Now that we have compiled the model, we can train it with the data we prepared earlier. The data to train with is specified with the first two parameters: `X_train` (input) and `y_train` (target). The data the fit function will validate the training data with is specified with the `validation_data` parameter.

The number of iterations that we will train the model for is specified with the `epochs` parameter. The `batch_size` specifies the number of training samples that are evaluated before the weights are updated in the network. Trial and error is the usual method for working out good values for the `epochs` and `batch_size`.

Sample output from the fit function is illustrated below. The output displays the average loss and accuracy of the training (loss, acc) and test dataset (val_loss, val_acc) for each epoch.

```
Train on 768 samples, validate on 254 samples
Epoch 1/150
768/768 [============] - 0s - loss: 5.1268 - acc: 0.6302 - val_loss: 4.9765 - val_acc: 0.6378
Epoch 2/150
768/768 [============] - 0s - loss: 4.9509 - acc: 0.6354 - val_loss: 4.8132 - val_acc: 0.6339
Epoch 3/150
```

In [8]:
model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=150, batch_size=10)

Train on 514 samples, validate on 254 samples
Epoch 1/150
514/514 [==============================] - 0s - loss: 0.6768 - acc: 0.6498 - val_loss: 0.6706 - val_acc: 0.6496
Epoch 2/150
514/514 [==============================] - 0s - loss: 0.6595 - acc: 0.6556 - val_loss: 0.6604 - val_acc: 0.6457
Epoch 3/150
514/514 [==============================] - 0s - loss: 0.6529 - acc: 0.6576 - val_loss: 0.6551 - val_acc: 0.6535
Epoch 4/150
514/514 [==============================] - 0s - loss: 0.6470 - acc: 0.6693 - val_loss: 0.6523 - val_acc: 0.6378
Epoch 5/150
514/514 [==============================] - 0s - loss: 0.6411 - acc: 0.6595 - val_loss: 0.6441 - val_acc: 0.6378
Epoch 6/150
514/514 [==============================] - 0s - loss: 0.6291 - acc: 0.6654 - val_loss: 0.6375 - val_acc: 0.6339
Epoch 7/150
514/514 [==============================] - 0s - loss: 0.6288 - acc: 0.6770 - val_loss: 0.6386 - val_acc: 0.6496
Epoch 8/150
514/514 [==============================] - 0s - loss: 0.6217 - acc: 0.6673

514/514 [==============================] - 0s - loss: 0.5253 - acc: 0.7451 - val_loss: 0.5703 - val_acc: 0.7323
Epoch 67/150
514/514 [==============================] - 0s - loss: 0.5236 - acc: 0.7374 - val_loss: 0.5731 - val_acc: 0.7362
Epoch 68/150
514/514 [==============================] - 0s - loss: 0.5278 - acc: 0.7412 - val_loss: 0.5806 - val_acc: 0.7205
Epoch 69/150
514/514 [==============================] - 0s - loss: 0.5264 - acc: 0.7451 - val_loss: 0.5870 - val_acc: 0.7244
Epoch 70/150
514/514 [==============================] - 0s - loss: 0.5360 - acc: 0.7354 - val_loss: 0.5694 - val_acc: 0.7244
Epoch 71/150
514/514 [==============================] - 0s - loss: 0.5272 - acc: 0.7393 - val_loss: 0.5704 - val_acc: 0.7283
Epoch 72/150
514/514 [==============================] - 0s - loss: 0.5245 - acc: 0.7374 - val_loss: 0.5768 - val_acc: 0.7441
Epoch 73/150
514/514 [==============================] - 0s - loss: 0.5272 - acc: 0.7354 - val_loss: 0.5682 - val_acc: 0.7441
Epoch 74/150


514/514 [==============================] - 0s - loss: 0.5072 - acc: 0.7568 - val_loss: 0.5628 - val_acc: 0.7402
Epoch 132/150
514/514 [==============================] - 0s - loss: 0.5070 - acc: 0.7315 - val_loss: 0.5636 - val_acc: 0.7441
Epoch 133/150
514/514 [==============================] - 0s - loss: 0.5049 - acc: 0.7510 - val_loss: 0.5621 - val_acc: 0.7480
Epoch 134/150
514/514 [==============================] - 0s - loss: 0.5025 - acc: 0.7432 - val_loss: 0.5689 - val_acc: 0.7323
Epoch 135/150
514/514 [==============================] - 0s - loss: 0.5110 - acc: 0.7529 - val_loss: 0.5622 - val_acc: 0.7362
Epoch 136/150
514/514 [==============================] - 0s - loss: 0.5036 - acc: 0.7432 - val_loss: 0.5608 - val_acc: 0.7402
Epoch 137/150
514/514 [==============================] - 0s - loss: 0.5008 - acc: 0.7549 - val_loss: 0.5596 - val_acc: 0.7441
Epoch 138/150
514/514 [==============================] - 0s - loss: 0.5061 - acc: 0.7549 - val_loss: 0.5725 - val_acc: 0.7559
Epoch 

### Evaluate the model
Now that we have trained our model, we can evaluate the performance on the test data prepared earlier. This is done with the `evaluate` function, simply supply the input and target parameters to the function.

The value printed out is the accuracy (%).

In [9]:
scores = model.evaluate(X, y)
print("\n{0}: {1:.2f}%".format(model.metrics_names[1], scores[1]*100))

 32/768 [>.............................] - ETA: 0s
acc: 75.00%
